In [2]:
import duckdb

In [3]:
#conexão com banco
con = duckdb.connect(database='dados_duckdb.db',read_only=False)

In [4]:
df = con.execute("""
                 SELECT * FROM 
                 (SELECT  *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY DT_INGESTAO DESC) as row
                 FROM bronze_z0019
                 WHERE DT_INGESTAO >= '2026-02-22')
                 WHERE row = 1
                 """).fetch_df()
df.head(10) #assim está pegando apenas o ultimo registro de cada NATBR

#dados duplicados do ID 10003 - PREGO vamos manter somente o estado final na Silver
#Como? de acordo com a data de ingestão sabemos qual foi o último a ser inserido.

,NATBR,MAKTX,WERKS,MAINS,LABST,NM_ARQUIVO,DT_INGESTAO,row
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-02-22 11:35:22.232150,1
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2026-02-22 11:35:22.232150,1
2,10003,PREGO,BT10,100,60,z0019_2.csv,2026-02-22 12:00:06.666797,1
3,10004,SERRA,BT50,100,200,z0019_2.csv,2026-02-22 12:00:06.666797,1
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2026-02-22 12:00:06.666797,1


In [10]:
#deletar colunas que não serão mais necessárias do dataframe
df_final = df.drop(columns=['NM_ARQUIVO','DT_INGESTAO','row'])

#renomear colunas de acordo com o glossário
df_final = df_final.rename(columns={'NATBR':'id', 'MAKTX':'nome', 'WERKS':'categoria'})
df_final = df_final.rename(columns={'MAINS':'fornecedor', 'LABST':'preco'})

#usar padrão de nomenclatura
df_final = df_final.rename(columns={'id':'ID_PRODUTO', 'nome':'NM_PRODUTO', 'categoria':'ID_CATEGORIA'})
df_final = df_final.rename(columns={'fornecedor':'ID_FORNECEDOR', 'preco':'VL_PRECO'})

df_final.head(10)

,ID_PRODUTO,NM_PRODUTO,ID_CATEGORIA,ID_FORNECEDOR,VL_PRECO
0,10001,PARAFUSO,BT10,100,100
1,10002,MARTELO,BT50,100,1500
2,10003,PREGO,BT10,100,60
3,10004,SERRA,BT50,100,200
4,10005,MACHADO,BT50,100,100


In [ ]:
#verificar qual o tipo de cada coluna -> está tudo como object (Strings)
df_final.dtypes

ID_PRODUTO       str
NM_PRODUTO       str
ID_CATEGORIA     str
ID_FORNECEDOR    str
VL_PRECO         str
dtype: object

In [ ]:
df2 = df_final
df2 = df2.astype(
    {
        'ID_PRODUTO':int,
        'NM_PRODUTO':str,
        'ID_CATEGORIA':str,
        'ID_FORNECEDOR':int,
        'VL_PRECO':float
    }
)

df2.dtypes

ID_PRODUTO         int64
NM_PRODUTO           str
ID_CATEGORIA         str
ID_FORNECEDOR      int64
VL_PRECO         float64
dtype: object

In [18]:
#criando tabela da camada silver
con.execute("""
    CREATE TABLE IF NOT EXISTS produtos(
            ID_PRODUTO BIGINT,
            NM_PRODUTO TEXT,
            ID_CATEGORIA TEXT,
            ID_FORNECEDOR BIGINT,
            VL_PRECO FLOAT
            )
""")

In [19]:
df2.head(10)

,ID_PRODUTO,NM_PRODUTO,ID_CATEGORIA,ID_FORNECEDOR,VL_PRECO
0,10001,PARAFUSO,BT10,100,100.0
1,10002,MARTELO,BT50,100,1500.0
2,10003,PREGO,BT10,100,60.0
3,10004,SERRA,BT50,100,200.0
4,10005,MACHADO,BT50,100,100.0


In [22]:
#inserção dos dados na tabela 
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [23]:
df_resultado = con.execute("SELECT * FROM produtos").fetch_df()
df_resultado.head(10)

,ID_PRODUTO,NM_PRODUTO,ID_CATEGORIA,ID_FORNECEDOR,VL_PRECO
0,10001,PARAFUSO,BT10,100,100.0
1,10002,MARTELO,BT50,100,1500.0
2,10003,PREGO,BT10,100,60.0
3,10004,SERRA,BT50,100,200.0
4,10005,MACHADO,BT50,100,100.0


In [24]:
con.close()